# Esper tutorial

This will walk you through a few of the APIs for accessing, visualizing, and processing data in the Esper environment.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Esper-tutorial" data-toc-modified-id="Esper-tutorial-1">Esper tutorial</a></span><ul class="toc-item"><li><span><a href="#Django-ORM-examples" data-toc-modified-id="Django-ORM-examples-1.1">Django ORM examples</a></span></li><li><span><a href="#Visualizing-video" data-toc-modified-id="Visualizing-video-1.2">Visualizing video</a></span></li><li><span><a href="#Querying-captions" data-toc-modified-id="Querying-captions-1.3">Querying captions</a></span></li><li><span><a href="#Extracting-audiovisual-data" data-toc-modified-id="Extracting-audiovisual-data-1.4">Extracting audiovisual data</a></span></li></ul></li></ul></div>

In [ ]:
import esper.prelude
from query.models import Video, Face
from esper.widget import esper_widget, qs_to_result, simple_result
from esper.captions import phrase_search

## Django ORM examples

We use Django to structure the code base, and we use its ORM to structure the data base. For example, we have a Video table that you can query using their ORM interface. In our dataset, each video is a 1-3 hour segment.

In [ ]:
Video.objects.all()[0]

Face bounding boxes are stored in the Face table. To see more tables, look at `app/query/models.py`

In [ ]:
Face.objects.all()[0]

The Django ORM converts Python-ish queries into SQL, e.g. here counting # of faces on CNN
(warning: this will take a while)

In [ ]:
Face.objects.filter(frame__video__channel__name='CNN').count()

## Visualizing video
To visualize results, we have a Jupyter-embedded widget that takes a data structure describing
a visualization. Here, we have the qs_to_result helper function that converts a Django QuerySet
into such a "reuslt" structure.

In [ ]:
qs_to_result(Face.objects.all(), limit=2)

We can use this structure with the esper_widget function.
Note that we use a number of keybindings to interact with the Esper widget. If you go to the 
main page at http://35.185.221.100, you can see extended help. You will have to click the
"disable jupyter keyboard" button while using the widget, since otherwise the keys conflict
with Jupyter's (sadly).

In [ ]:
widget = esper_widget(qs_to_result(Face.objects.all()))
widget

If you select clips in the widget by clicking "s" on the clip, then the indices of the selected clips
are available in Python here. This is useful for doing labeling tasks inside a Jupyter notebook.

In [ ]:
widget.selected

## Querying captions
We have Python libraries for searching through our captions. This one returns all instances of a particular
phrase in the captions. The source files are located at `app/data/subs/orig`

In [ ]:
generator = phrase_search('MERRY CHRISTMAS')
phrase_instance = next(generator)
phrase_instance

Show the video clip corresponding to the phrase. Press "p" to play.

In [ ]:
esper_widget(simple_result([{
    'video': phrase_instance.id,
    'min_time': phrase_instance.postings[0].start,
    'max_time': phrase_instance.postings[0].end
}]))

## Extracting audiovisual data
The video files are too big to be stored on any machine (unlike the captions). They're on Google Cloud Storage, at `gs://esper/tvnews/videos`. However, we have a number of APIs for accessing raw data. For example, we use the [`scannertools`](https://github.com/scanner-research/scannertools) API to access individual frames of a video.

In [ ]:
import matplotlib.pyplot as plt
video = Video.objects.all()[0]
plt.imshow(video.for_scannertools().frame(100))

You can also use ffmpeg to download segments of video or audio.

In [ ]:
print(video.download(ext='aac', segment=(0, 10)))  # audio from time 0s to 10s
print(video.download(ext='mp4', segment=(10, 20))) # video+audio from time 10s to 20s